# IAU Projekt 3. fáza

### Autori: Martin Raffáč (50%), Tibor Vanek (50%)

dataset číslo **003**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree, metrics
from pandas import read_csv
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV
from matplotlib import pyplot
import os
#import graphviz 
#import pydot
from IPython.display import Image
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import validation_curve
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.feature_selection import VarianceThreshold

In [39]:
train_data = pd.read_csv("num_data.csv", sep=',')
test_data = pd.read_csv("test_data.csv", sep=',')

Náčítanie trénovacích dát.

In [40]:
data_for_oneR = train_data.copy()

## 1. Jednoduchý klasifikátor na základe závislosti v dátach 

In [41]:
def oneR(column):

    split = (data_for_oneR[column][data_for_oneR['warning'] == 0].mean() + data_for_oneR[column][data_for_oneR['warning'] == 1].mean())/2
    group = data_for_oneR[data_for_oneR[column] > split]
    group = group.groupby('warning').size()

    if group[0] > group[1]:
        left = 1
        right = 0
    else:
        left = 0
        right = 1
        
    predict = {"split": split, "left": left, "right": right}
    
    data_for_oneR.loc[data_for_oneR[column] < predict["split"], 'helper'] = predict["left"]
    data_for_oneR.loc[data_for_oneR[column] >= predict["split"], 'helper'] = predict["right"]
    accuracy = accuracy_score(data_for_oneR['warning'], data_for_oneR['helper'])
    precision = precision_score(data_for_oneR['warning'], data_for_oneR['helper'])
    recall = recall_score(data_for_oneR['warning'], data_for_oneR['helper'])
    total=(accuracy + precision + recall) / 3
    data_for_oneR.drop('helper', axis=1, inplace=True)

    return {"accuracy": accuracy, "precision": precision, "recall": recall ,"total": total}

In [42]:
accuracy = 0 
columns =  data_for_oneR.columns

for i in columns:
    if i == 'warning':
        continue

    data_oneR = oneR(i)
    
    if data_oneR["total"] > accuracy:
        best = i
        accuracy = data_oneR["total"]
        best_column = data_oneR
        
print('best oneR: ' + best)
print('total_score: ' + str(best_column["total"]))
print('accuracy_score: ' + str(best_column["accuracy"]))
print('precision_score: ' + str(best_column["precision"]))
print('recall_score: ' + str(best_column["recall"]))

best oneR: PAHs
total_score: 0.8226555439842604
accuracy_score: 0.8033508691674291
precision_score: 0.8549149813864574
recall_score: 0.8097007813988946


Podľa zadania sme zhotovili jednoduchý algoritmus oneR, ktorý sme vložili do funkcie. Následne sme ho vyskúšali na trénovacom datasete, predpokladali sme, že najlepší výsledok dosiahne atrihút PAHs, pretože má najviššiu koreláciu s atribútom warning. Ako je možné vidieť vyššie naše predpoklady sa naplinili.  

## 2. Trénovanie a vyhodnotenie klasifikátorov strojového učenia

In [43]:
# treba trochu precistit dataset
train_data.drop(["Unnamed: 0"], axis= "columns", inplace = True)
test_data.drop(["Unnamed: 0", "station", "QoS", "code", "revision"], axis= "columns", inplace = True)

In [44]:
train_data.head()

,longitude,C2H3NO5,O3,CO,warning,Pb,PAHs,PM2.5,PRES,TEMP,latitude,NH3,H2CO,SO2,CH4,PM10,CFCs,NOx
0,97.01124,-1.318894,1.427437,1.162054,1.0,1.954680,-1.005176,-1.709415,1144.53017,0.676926,17.30858,0.252756,0.865279,0.924576,0.328161,-2.171795,-0.266693,-2.321369
1,126.83500,-2.037985,-0.577099,-0.093487,0.0,1.541669,-0.518571,-0.160529,1174.97809,1.670039,37.65639,-0.645434,-0.526938,-1.022389,-0.772874,-0.374415,-1.957799,2.060238
2,-77.43532,0.858519,-0.836656,-0.977796,0.0,1.138531,-1.155592,0.775651,1091.03764,0.555705,20.14298,0.215612,-0.194127,-0.467293,-0.997094,1.110119,0.932770,2.259446
3,24.43973,-1.001980,2.042868,-0.461678,0.0,0.062939,-0.511242,-1.970697,1069.54392,1.507952,0.76755,-1.170966,0.018366,0.469582,0.147670,-1.569419,-1.494582,1.120510
4,70.91106,-0.160618,0.045666,-0.137935,1.0,1.371373,0.253984,-0.592472,1084.32069,0.345856,29.38242,0.619865,-1.107548,0.946049,1.618434,-0.321605,-0.281218,-0.796831


In [45]:
test_data.head()

,longitude,C2H3NO5,O3,CO,warning,Pb,PAHs,PM2.5,PRES,TEMP,latitude,NH3,H2CO,SO2,CH4,PM10,CFCs,NOx
0,-63.13414,0.03970,9.20861,7.59667,0.0,41.86251,7.80967,6.27422,1124.957487,13.21464,46.23899,5.80241,43.22794,9.72249,10.43725,7.65940,53.79674,8.81237
1,-76.99081,0.08607,8.92422,6.89382,1.0,48.26615,10.03832,7.67315,1140.626970,12.79735,38.96372,8.21937,50.62776,9.09776,7.52423,8.69599,49.72505,6.72095
2,-65.71875,0.02946,6.87142,7.62699,1.0,54.26844,9.43831,8.87614,1182.998540,8.30216,-21.44345,9.91387,53.58158,8.74041,10.58385,6.01329,69.02005,6.84437
3,145.06667,0.03215,7.58483,9.64289,1.0,64.83627,8.41452,7.04136,1093.056920,25.42489,-37.88333,6.98521,25.74717,10.08661,10.25194,5.24794,54.31056,6.09566
4,136.51667,0.02341,7.28777,8.99423,0.0,56.46623,7.46872,6.38319,1155.669550,30.92183,35.00000,7.34623,33.30239,7.48142,8.20504,6.48610,61.45096,9.25059


Stĺpec warning si oddelíme od datasetu do samostatných premenných

In [139]:
train_target = train_data["warning"]
test_target = test_data["warning"]

#use_train_data = train_data.drop(['warning'], axis = "columns")
#use_test_data = test_data.drop(['warning'], axis = "columns")

x_train_data = train_data[['CH4', 'O3', 'PM2.5', 'SO2', 'longitude', 'latitude', 'C2H3NO5', 'CO', 'NH3', 'NOx']]
x_test_data = test_data[['CH4', 'O3', 'PM2.5', 'SO2', 'longitude', 'latitude', 'C2H3NO5', 'CO', 'NH3', 'NOx']]
# vybrane atributy podla WAQI indexu a najvyssich korelacii

In [144]:
dtf = tree.DecisionTreeClassifier()
dtf.fit(x_train_data, train_target)
prediction = dtf.predict(x_test_data)
print("Score: ", dtf.score(x_test_data, test_target))

Score:  0.6044029352901935


In [145]:
print(classification_report(test_target, prediction, target_names = ["0","1"]))

print("Accuracy:", accuracy_score(test_target, prediction))
print("Precision:", precision_score(test_target, prediction))
print("Recall:", recall_score(test_target, prediction))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00      2967
           1       0.60      1.00      0.75      4528

    accuracy                           0.60      7495
   macro avg       0.80      0.50      0.38      7495
weighted avg       0.76      0.60      0.46      7495

Accuracy: 0.6044029352901935
Precision: 0.6042973441879087
Recall: 1.0


In [111]:
# graphviz nefungoval, toto je prilis velky obrazok
#fig = plt.figure(figsize = (15, 20))
#_ = tree.plot_tree(dtf, feature_names = use_test_data.columns, class_names = ["1","0"], filled = True)
#fig.savefig("strom.png")

## 3. Optimalizácia alias hyperparameter tuning

Preskúmanie hyperparametrov

1. criterion je funkcia, ktorá meria kvalitu rozdelenia. Obsahuje hodnoty gini a entropy

2. splitter je stratégia rozdelenia, ktorá sa používa na všetkých uzloch stromu. využíva stratégie best a random.

3. max_depth je maximalna hĺbka stromu.

4. max_features charakterizuje počet stĺpcov, ktoré sa berú na vedomie pri hľadaní najlepšieho rozdelenia. Do max_depth a max_features sme vložili rozsah hodnôt.

In [185]:
parameters = {'criterion': ('gini','entropy'), 'splitter': ('best','random'), 'max_features':range(1,20,1)}
optimization = GridSearchCV(dtf, parameters,cv=5,scoring="accuracy")
result = optimization.fit(x_train_data, train_target)

Na vytvorenie modelu pre všetky kombinácie sme využili metódu GridSearchCV() 

In [186]:
params = optimization.best_params_
params

{'criterion': 'entropy', 'max_features': 6, 'splitter': 'best'}

In [187]:
tab = pd.DataFrame(optimization.cv_results_)
tab.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019008,0.004897,0.003000,0.006000,gini,1,best,"{'criterion': 'gini', 'max_features': 1, 'spli...",0.912236,0.927673,0.911950,0.925079,0.917358,0.918859,0.006484,72
1,0.009006,0.007354,0.003000,0.005999,gini,1,random,"{'criterion': 'gini', 'max_features': 1, 'spli...",0.881933,0.895083,0.886221,0.895053,0.912496,0.894157,0.010492,76
2,0.030051,0.000828,0.000408,0.000815,gini,2,best,"{'criterion': 'gini', 'max_features': 2, 'spli...",0.934820,0.939680,0.937679,0.949385,0.940520,0.940417,0.004895,56
3,0.008572,0.003230,0.003675,0.003688,gini,2,random,"{'criterion': 'gini', 'max_features': 2, 'spli...",0.903087,0.907947,0.913665,0.927938,0.926508,0.915829,0.009898,74
4,0.040853,0.000027,0.000424,0.000848,gini,3,best,"{'criterion': 'gini', 'max_features': 3, 'spli...",0.941109,0.943682,0.945397,0.951101,0.944810,0.945220,0.003288,34


In [188]:
opt = optimization.best_estimator_
opt

DecisionTreeClassifier(criterion='entropy', max_features=6)

In [189]:
clf = optimization.best_estimator_
clf = clf.fit(x_train_data,train_target)

predict=clf.predict(x_test_data)

In [190]:
print("score: ",metrics.accuracy_score(test_target,predict))

score:  0.6041360907271515
